<a href="https://colab.research.google.com/github/ZzangJay/team_project/blob/kimjisan/220921_prescriptions_ab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Prescription 에서
###항생제 처방 추출
#####-투여경로: IV, PO 등(점이,점안 등은 제외)
#####-이름에 ointment, cream, gel, desensitization 등 포함 제외

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
prescriptions=pd.read_csv('/content/drive/MyDrive/mimiciv_unzip/hosp/prescriptions.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
prescriptions.columns

Index(['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 'poe_seq',
       'starttime', 'stoptime', 'drug_type', 'drug', 'formulary_drug_cd',
       'gsn', 'ndc', 'prod_strength', 'form_rx', 'dose_val_rx', 'dose_unit_rx',
       'form_val_disp', 'form_unit_disp', 'doses_per_24_hrs', 'route'],
      dtype='object')

In [4]:
prescriptions['drug']=prescriptions['drug'].astype(str)
prescriptions['drug']=prescriptions['drug'].map(str.lower) #모두 소문자로 바꿈

In [5]:
# prescriptions[prescriptions['drug'].str.contains('gentamicin')==True]['drug']

In [6]:
antbi=['adoxa', 'ala-tet', 'alodox', 'amikacin', 'amikin', 'amoxicillin', 'amoxicillin%clavulanate', 'clavulanate', 'ampicillin',
       'augmentin', 'avelox', 'avidoxy', 'azactam', 'azithromycin', 'aztreonam', 'axetil', 'bactocill', 'bactrim', 'bethkis',
       'biaxin', 'bicillin l-a', 'cayston', 'cefazolin', 'cedax', 'cefoxitin', 'ceftazidime', 'cefaclor', 'cefadroxil', 'cefdinir',
       'cefditoren', 'cefepime', 'cefotetan', 'cefotaxime', 'cefpodoxime', 'cefprozil', 'ceftibuten', 'ceftin', 'cefuroxime ', 
       'cefuroxime', 'cephalexin', 'chloramphenicol', 'cipro', 'ciprofloxacin', 'claforan', 'clarithromycin', 'cleocin', 'clindamycin', 
       'cubicin', 'dicloxacillin', 'doryx', 'doxycycline', 'duricef', 'dynacin', 'ery-tab', 'eryped', 'eryc', 'erythrocin', 'erythromycin', 
       'factive', 'flagyl', 'fortaz', 'furadantin', 'garamycin', 'gentamicin', 'kanamycin', 'keflex', 'ketek', 'levaquin', 'levofloxacin', 
       'lincocin', 'macrobid', 'macrodantin', 'maxipime', 'mefoxin', 'metronidazole', 'minocin', 'minocycline', 'monodox', 'monurol', 
       'morgidox', 'moxatag', 'moxifloxacin', 'myrac', 'nafcillin sodium', 'nicazel doxy 30', 'nitrofurantoin', 'noroxin', 'ocudox', 
       'ofloxacin', 'omnicef', 'oracea', 'oraxyl', 'oxacillin', 'pc pen vk', 'pce dispertab', 'panixine', 'pediazole', 'penicillin', 
       'periostat', 'pfizerpen', 'piperacillin', 'tazobactam', 'primsol', 'proquin', 'raniclor', 'rifadin', 'rifampin', 'rocephin', 'smz-tmp', 
       'septra', 'septra ds', 'septra', 'solodyn', 'spectracef', 'streptomycin sulfate', 'sulfadiazine', 'sulfamethoxazole', 'trimethoprim', 
       'sulfatrim', 'sulfisoxazole', 'suprax', 'synercid', 'tazicef', 'tetracycline', 'timentin', 'tobi', 'tobramycin', 'trimethoprim', 
       'unasyn', 'vancocin', 'vancomycin', 'vantin', 'vibativ', 'vibra-tabs', 'vibramycin', 'zinacef', 'zithromax', 'zmax', 'zosyn', 'zyvox']

In [7]:
# antbi 리스트에 있는 항생제 처방기록만 추출

ab_list_patt = '|'.join(antbi)
prescriptions_ab = prescriptions[prescriptions['drug'].str.contains(ab_list_patt)==True]

In [8]:
prescriptions_ab

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,starttime,stoptime,drug_type,drug,formulary_drug_cd,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
25,10000032,22841357,53555857,10000032-94,94.0,2180-06-26 23:00:00,2180-06-27 23:00:00,MAIN,sulfameth/trimethoprim ds,BACTDS,009396,9.042726e+08,DS Tablet,NaN,1,TAB,1,TAB,1.0,PO/NG
115,10000117,27988844,52240876,10000117-119,119.0,2183-09-19 12:00:00,2183-09-20 15:00:00,MAIN,cefazolin,CEFA2D100,075120,3.383508e+08,2 g / 100 mL Dextrose (iso-os),NaN,2,g,1,BAG,3.0,IV
177,10000724,20823482,5703283,10000724-101,101.0,2145-04-25 00:00:00,2145-04-28 20:00:00,MAIN,cefazolin,CEFA1F,051880,2.643103e+08,1g Froz.Bag,NaN,1,g,1,BAG,3.0,IV
221,10000764,27897940,67322733,10000764-50,50.0,2132-10-15 20:00:00,2132-10-19 00:00:00,MAIN,amoxicillin-clavulanic acid,AUGM500,008992,6.668510e+10,500mg Tablet,NaN,500,mg,1,TAB,3.0,PO/NG
262,10000826,20032235,52793571,10000826-121,121.0,2146-12-09 10:00:00,2146-12-12 13:00:00,MAIN,ciprofloxacin hcl,CIPR500,009510,1.725312e+08,500mg Tab,NaN,500,mg,1,TAB,2.0,PO/NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16219304,19999840,21033226,84028816,19999840-239,239.0,2164-09-15 12:00:00,2164-09-16 12:00:00,MAIN,piperacillin-tazobactam na,ZOSY4FPB,040819,2.068862e+08,4.5 g Frozen Bag,NaN,4.5,g,1,BAG,3.0,IV
16219319,19999840,26071774,12714371,19999840-54,54.0,2164-07-26 20:00:00,2164-07-26 13:00:00,MAIN,sulfameth/trimethoprim ds,BACTDS,009396,9.042726e+08,DS Tablet,NaN,1,TAB,1,TAB,2.0,PO
16219369,19999987,23865745,57653807,19999987-196,196.0,2145-11-08 20:00:00,2145-11-09 15:00:00,MAIN,vancomycin,VANC1F,043952,3.383552e+08,1g Frozen Bag,NaN,1000,mg,1,BAG,2.0,IV
16219375,19999987,23865745,61631314,19999987-210,210.0,2145-11-09 20:00:00,2145-11-11 17:00:00,MAIN,sulfameth/trimethoprim ds,BACTDS,009396,9.042726e+08,DS Tablet,NaN,1,TAB,1,TAB,2.0,PO/NG


In [9]:
prescriptions_ab['route'].value_counts()

IV              471350
PO/NG           147962
PO               57958
BOTH EYES         2421
TP                2038
RIGHT EYE         1169
LEFT EYE          1143
LOCK              1091
PB                 805
PR                 701
IP                 644
ORAL               499
OU                 486
NG                 462
IH                 272
RIGHT EAR          235
LEFT EAR           220
OD                 197
BOTH EARS          160
PL                 152
OS                 127
VG                 118
IM                 113
IRR                 29
INTRAVITREAL        21
IVT                 19
IT                  12
AD                  11
AS                   8
DWELL                8
NEB                  5
AU                   5
J TUBE               4
G TUBE               3
INTERSPACE           2
INHALATION           2
INTRAPLEURAL         2
IV BOLUS             2
NAS                  2
IJ                   1
IN                   1
IL                   1
IV DRIP              1
EX-VIVO    

### 투여경로 short name 참고 

https://www.fda.gov/drugs/data-standards-manual-monographs/route-administration

In [10]:
route_all=prescriptions_ab['route'].unique()
print(route_all)

['PO/NG' 'IV' 'PO' 'ORAL' 'RIGHT EYE' 'TP' 'OU' 'LEFT EYE' 'BOTH EYES'
 'PR' 'VG' 'NG' 'IM' 'OS' 'PB' 'LEFT EAR' 'RIGHT EAR' 'PL' 'IP' 'LOCK'
 'OD' 'INHALATION' 'BOTH EARS' 'INTRAVITREAL' 'AD' 'IH' 'NEB' 'IRR'
 'INTERSPACE' 'AS' nan 'AU' 'DWELL' 'IVS' 'IVT' 'IT' 'J TUBE' 'IJ'
 'INTRAPLEURAL' 'IV BOLUS' 'NAS' 'IN' 'IL' 'IV DRIP' 'EX-VIVO' 'G TUBE']


In [11]:
#투여경로 
# eyes, ears, topical 등 제외 
route_list=['PO/NG','IV','PO','ORAL','NG','IM','IP','IV BOLUS']
prescriptions_ab=prescriptions_ab[prescriptions_ab['route'].isin(route_list)]
prescriptions_ab

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,starttime,stoptime,drug_type,drug,formulary_drug_cd,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
25,10000032,22841357,53555857,10000032-94,94.0,2180-06-26 23:00:00,2180-06-27 23:00:00,MAIN,sulfameth/trimethoprim ds,BACTDS,009396,9.042726e+08,DS Tablet,NaN,1,TAB,1,TAB,1.0,PO/NG
115,10000117,27988844,52240876,10000117-119,119.0,2183-09-19 12:00:00,2183-09-20 15:00:00,MAIN,cefazolin,CEFA2D100,075120,3.383508e+08,2 g / 100 mL Dextrose (iso-os),NaN,2,g,1,BAG,3.0,IV
177,10000724,20823482,5703283,10000724-101,101.0,2145-04-25 00:00:00,2145-04-28 20:00:00,MAIN,cefazolin,CEFA1F,051880,2.643103e+08,1g Froz.Bag,NaN,1,g,1,BAG,3.0,IV
221,10000764,27897940,67322733,10000764-50,50.0,2132-10-15 20:00:00,2132-10-19 00:00:00,MAIN,amoxicillin-clavulanic acid,AUGM500,008992,6.668510e+10,500mg Tablet,NaN,500,mg,1,TAB,3.0,PO/NG
262,10000826,20032235,52793571,10000826-121,121.0,2146-12-09 10:00:00,2146-12-12 13:00:00,MAIN,ciprofloxacin hcl,CIPR500,009510,1.725312e+08,500mg Tab,NaN,500,mg,1,TAB,2.0,PO/NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16219304,19999840,21033226,84028816,19999840-239,239.0,2164-09-15 12:00:00,2164-09-16 12:00:00,MAIN,piperacillin-tazobactam na,ZOSY4FPB,040819,2.068862e+08,4.5 g Frozen Bag,NaN,4.5,g,1,BAG,3.0,IV
16219319,19999840,26071774,12714371,19999840-54,54.0,2164-07-26 20:00:00,2164-07-26 13:00:00,MAIN,sulfameth/trimethoprim ds,BACTDS,009396,9.042726e+08,DS Tablet,NaN,1,TAB,1,TAB,2.0,PO
16219369,19999987,23865745,57653807,19999987-196,196.0,2145-11-08 20:00:00,2145-11-09 15:00:00,MAIN,vancomycin,VANC1F,043952,3.383552e+08,1g Frozen Bag,NaN,1000,mg,1,BAG,2.0,IV
16219375,19999987,23865745,61631314,19999987-210,210.0,2145-11-09 20:00:00,2145-11-11 17:00:00,MAIN,sulfameth/trimethoprim ds,BACTDS,009396,9.042726e+08,DS Tablet,NaN,1,TAB,1,TAB,2.0,PO/NG


In [12]:
# 항생제 중 ointment, cream, gel, desensitization 등 제외

drug_excl=['cream','oint','gel','ophth','desens']

drug_excl_patt = '|'.join(drug_excl)
prescriptions_ex = prescriptions_ab[prescriptions_ab['drug'].str.contains(drug_excl_patt)==True]
prescriptions_ab = prescriptions_ab[prescriptions_ab['drug'].str.contains(drug_excl_patt)==False]

print(prescriptions_ex)

          subject_id   hadm_id  pharmacy_id         poe_id  poe_seq  \
28932       10017531  21095812      5251146  10017531-1386   1386.0   
28948       10017531  21095812     12067018  10017531-1384   1384.0   
28974       10017531  21095812     22181365  10017531-1388   1388.0   
29001       10017531  21095812     29946921  10017531-1387   1387.0   
29061       10017531  21095812     47777133  10017531-1387   1387.0   
...              ...       ...          ...            ...      ...   
16027215    19883311  23262610     57047569  19883311-1543   1543.0   
16027222    19883311  23262610     64655372  19883311-1542   1542.0   
16027225    19883311  23262610     67734582  19883311-1541   1541.0   
16027227    19883311  23262610     67782499  19883311-1534   1534.0   
16217719    19999297  21439025     19651826   19999297-274    274.0   

                    starttime             stoptime drug_type  \
28932     2160-01-03 16:00:00  2160-01-03 16:00:00      MAIN   
28948     2160-01-0

In [13]:
print(prescriptions_ab)

          subject_id   hadm_id  pharmacy_id        poe_id  poe_seq  \
25          10000032  22841357     53555857   10000032-94     94.0   
115         10000117  27988844     52240876  10000117-119    119.0   
177         10000724  20823482      5703283  10000724-101    101.0   
221         10000764  27897940     67322733   10000764-50     50.0   
262         10000826  20032235     52793571  10000826-121    121.0   
...              ...       ...          ...           ...      ...   
16219304    19999840  21033226     84028816  19999840-239    239.0   
16219319    19999840  26071774     12714371   19999840-54     54.0   
16219369    19999987  23865745     57653807  19999987-196    196.0   
16219375    19999987  23865745     61631314  19999987-210    210.0   
16219381    19999987  23865745     69924281  19999987-161    161.0   

                    starttime             stoptime drug_type  \
25        2180-06-26 23:00:00  2180-06-27 23:00:00      MAIN   
115       2183-09-19 12:00:00  